In [ ]:
# | default_exp cli

# Generate_Docs_CLI
> CLI commands to generate documentaion

In [ ]:
# | export

from typing import *
import typer

import types

import pkgutil
from pathlib import Path
import shutil
import os
import importlib

In [ ]:
import tempfile
from typer.testing import CliRunner

In [ ]:
runner = CliRunner()

In [ ]:
# | export

app = typer.Typer(help="CLI commands for generating mkdocs-material documentation")

## Generate summary

In [ ]:
# | export


def list_submodules(package_name: types.ModuleType) -> List[str]:
    def _list_submodules(list_name, package_name):
        for loader, module_name, is_pkg in pkgutil.walk_packages(
            package_name.__path__, package_name.__name__ + "."
        ):
            if (
                not module_name.split(".")[1].startswith("_")
                and not module_name.split(".")[1] == "mkdocs"
            ):
                list_name.append(module_name)
                module_name = __import__(module_name, fromlist="dummylist")
                if is_pkg:
                    _list_submodules(list_name, module_name)

    all_modules: List[str] = []
    _list_submodules(all_modules, package_name)

    return sorted(set(all_modules))

In [ ]:
import nbdev_mkdocs

airt_modules = list_submodules(nbdev_mkdocs)
airt_modules

['nbdev_mkdocs.cli',
 'nbdev_mkdocs.docstring',
 'nbdev_mkdocs.docstring.helpers']

In [ ]:
# | export


def get_exported_symbols(submodules: List[str]) -> Dict[str, List[str]]:
    modules = {name: __import__(name, fromlist="dummylist") for name in submodules}
    return {
        name: module.__all__
        for name, module in modules.items()
        if hasattr(module, "__all__")
    }

In [ ]:
symbols = get_exported_symbols(airt_modules)
symbols

{'nbdev_mkdocs.cli': ['app',
  'list_submodules',
  'get_exported_symbols',
  'generate_nav_tree',
  'build_api_docs'],
 'nbdev_mkdocs.docstring.helpers': ['run_examples_from_docstring']}

In [ ]:
# | export


def generate_nav_tree(level, summary, m, all_modules):
    """Function to generate multi-level natigation in the docs"""
    base_module = ".".join(m.split(".")[: -(level - 1)])
    sub_module_1 = ".".join(m.split(".")[2 : -(level - 2)]) if level > 2 else None
    sub_module_2 = ".".join(m.split(".")[3 : -(level - 3)]) if level > 3 else None
    leaf_sub_module = m.split(".")[-1]

    if base_module not in all_modules:
        all_modules.add(base_module)
        _indent = "    " * (2) if level == 3 else "    "
        summary = summary + f"{_indent}- {base_module}\n"

    if (sub_module_1) and (sub_module_1 not in all_modules):
        all_modules.add(sub_module_1)
        _indent = "    " * (3) if level == 3 else "    " * (2)
        summary = summary + f"{_indent}- {sub_module_1}\n"

    if (sub_module_2) and (sub_module_2 not in all_modules):
        all_modules.add(sub_module_2)
        summary = summary + f"{'    '*(level-1)}- {sub_module_2}\n"

    summary = summary + f"{'    '*level}- {leaf_sub_module}\n"

    return summary, all_modules

In [ ]:
# | export

_summary = """
- [Home](index.md)
"""

# typer
# def build_api_docs(module = airt, summary: str = _summary, docs_path: Path = Path("docs")):
def build_api_docs(
    module: str = typer.Argument(..., help="The module name to generate docs"),
    docs_path: str = typer.Option(default="docs"),
):
    _docs_path = Path(str(docs_path))  # type: ignore

    #     module = airt
    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    imported_module = importlib.import_module(module)
    summary = _summary

    submodules = list_submodules(imported_module)
    symbols = get_exported_symbols(submodules)

    if not _docs_path.exists():
        raise ValueError(f"{_docs_path=}, {os.getcwd()=}")

    shutil.rmtree(_docs_path / "API", ignore_errors=True)

    summary = summary + f"- API\n"
    all_modules: Set[str] = set()

    for m, sx in symbols.items():

        level = len(m.split(".")) - 1

        parts = m.split(".")[-1:]
        path_link = "API/" + "/".join(parts)
        path = _docs_path / path_link

        if level == 1:
            summary = summary + f"{'    '*(level)}- {m}\n"
        else:
            summary, all_modules = generate_nav_tree(level, summary, m, all_modules)

        path.mkdir(exist_ok=True, parents=True)

        for s in sx:
            summary = summary + f"{'    '*(level+1)}- [{s}]({path_link}/{s}.md)\n"
            fname = s + ".md"

            with open(path / fname, mode="w") as f:
                print(f"Writing {Path(path_link) / fname}")
                f.write(f"#{s}\n::: {m}.{s}\n")

    #     summary = summary + "- [Release](RELEASE.md)"

    with open(_docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)

    return summary


# print(build_api_docs(docs_path = "../docs"))

In [ ]:
with tempfile.TemporaryDirectory() as d:
    docs_path = Path(d) / "docs"
    docs_path.mkdir(parents=True)
    #     shutil.copytree("../docs", docs_path)
    build_api_docs(module="nbdev_mkdocs", docs_path=docs_path)

#     !cat {docs_path}/SUMMARY.md

Writing API/cli/app.md
Writing API/cli/list_submodules.md
Writing API/cli/get_exported_symbols.md
Writing API/cli/generate_nav_tree.md
Writing API/cli/build_api_docs.md
Writing API/helpers/run_examples_from_docstring.md


In [ ]:
# | export


app.command()(build_api_docs)

<function __main__.build_api_docs(module: str = <typer.models.ArgumentInfo object>, docs_path: str = <typer.models.OptionInfo object>)>

In [ ]:
result = runner.invoke(app, ["--help"])

print(result.stdout)

                                                                                                                   
 Usage: build-api-docs [OPTIONS] MODULE                                                                            
                                                                                                                   

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    module      TEXT  The module name to generate docs [default: None] [required]                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --docs-path                 TEXT  [default: docs]                                                               │
│ --install-completion              Install completion for the current shell.                                     │
│ --show-completion                 Show completion for the current shell, to copy it or customize the            │
│                                   installation.                                                                 │
│ --help                            Show this message and exit.                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯